# Microbot Feeling

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=GOOGLE_API_KEY)



/home/arrcus/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import re


# Initializing emotional scales
sad_value = 0
angry_value = 0

# Prompt Templates
happy_sad_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are a sentiment analyzer. Rate the following human input on a scale from -3 (very sad) to 3 (very happy). The scale represents the emotional content. \
    -3 indicates a very sadening to hear; \
    -2 indicates a sadening to hear; \
    -1 indicates a little bit sadening to hear; \
    0 indicates a neutral sentiment; \
    1 indicates a little bit making happy to hear; \
    2 indicates a making happy to hear; \
    3 indicates a very making happy to hear. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

calm_angry_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are an emotion analyzer. Rate the following human input on a scale from -3 (very calm) to 3 (very angry). The scale represents the emotion expressed. \
    -3 indicates a very calm tone; \
    -2 indicates a calm tone; \
    -1 indicates a slightly calm tone; \
    0 indicates a neutral tone; \
    1 indicates a slightly angry tone; \
    2 indicates an angry tone; \
    3 indicates a very angry tone. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

robot_command_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are a robot command analyzer. Determine if the following human input contains a robot movement command. \
    A valid command is in the format '[x]:[y]', where 'x' and 'y' are numbers. \
    If a valid command is present, extract and return only that command as a string. If no valid command is present return the string 'no command'. \
    Input: {human_input}"
)


# 3. LLM Chains
happy_sad_chain = LLMChain(llm=llm, prompt=happy_sad_prompt)
calm_angry_chain = LLMChain(llm=llm, prompt=calm_angry_prompt)
robot_command_chain = LLMChain(llm=llm, prompt=robot_command_prompt)

def rate_sentiment(text, chain, scale_name, current_value):
    """Rates the sentiment of a given text using a Gemini Pro model.
    Args:
        text (str): The text to rate.
        chain (LLMChain): The LLMChain to use
        scale_name (str): The name of the scale to rate
        current_value (int): The current value for this scale
    Returns:
        tuple(str,int): A tuple containing a single number (rating) representing the sentiment between -10 and 10, or "error" if the rating was not in the range
        and the new value of this scale
    """

    result = chain.run(human_input=text)
    
    try:
      rating = int(result.strip())
      if -10 <= rating <= 10:
        current_value += rating
        current_value = max(-10, min(current_value, 10))
        return str(rating), current_value
      else:
        return "error", current_value
    except ValueError:
      return "error", current_value
    
def extract_command(text, chain):
    """
    Extracts a robot command from the given text.
    Args:
        text (str): The text to analyze.
    Returns:
        str: the robot command if found or "no command"
    """
    result = chain.run(human_input=text)

    
    return result.strip()

if __name__ == "__main__":
    while True:
        user_input = input("Enter some text (or 'exit' to quit): ")
        if user_input.lower() == "exit":
            break
        
        happy_sad_rating, sad_value = rate_sentiment(user_input, happy_sad_chain, "happy-sad", sad_value)
        if happy_sad_rating != "error":
            print(f"Sentiment rating (happy-sad): {happy_sad_rating}")
            print(f"Sadness-Happiness Scale: {sad_value}")
        else:
            print(f"Error: Couldn't rate the sentiment of this input.")
        
        calm_angry_rating, angry_value = rate_sentiment(user_input, calm_angry_chain, "calm-angry", angry_value)
        if calm_angry_rating != "error":
          print(f"Emotion rating (calm-angry): {calm_angry_rating}")
          print(f"Anger Scale: {angry_value}")
        else:
            print(f"Error: Couldn't rate the emotion of this input.")
        if sad_value > 5 or angry_value > 5:
            robot_command = extract_command(user_input, robot_command_chain)
            print(f"Robot Command: {robot_command}")
        else:
           print("Too Sad to move")

# With Streamlit App

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
import streamlit as st

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# 1. LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=GOOGLE_API_KEY)

# Initializing emotional scales
sad_value = 0
angry_value = 0

# 2. Prompt Templates
happy_sad_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are a sentiment analyzer. Rate the following human input on a scale from -3 (very sad) to 3 (very happy). The scale represents the emotional content. \
    -3 indicates a very sadening to hear; \
    -2 indicates a sadening to hear; \
    -1 indicates a little bit sadening to hear; \
    0 indicates a neutral sentiment; \
    1 indicates a little bit making happy to hear; \
    2 indicates a making happy to hear; \
    3 indicates a very making happy to hear. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

calm_angry_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are an emotion analyzer. Rate the following human input on a scale from -3 (very calm) to 3 (very angry). The scale represents the emotion expressed. \
    -3 indicates a very calm tone; \
    -2 indicates a calm tone; \
    -1 indicates a slightly calm tone; \
    0 indicates a neutral tone; \
    1 indicates a slightly angry tone; \
    2 indicates an angry tone; \
    3 indicates a very angry tone. \
    Your answer must be a single number between -3 and 3: {human_input}"
)

robot_command_prompt = PromptTemplate(
    input_variables=["human_input"],
    template="You are a robot command analyzer. Determine if the following human input contains a robot movement command. \
    A valid command is in the format '[x]:[y]', where 'x' and 'y' are numbers. \
    If a valid command is present, extract and return only that command as a string. If no valid command is present return the string 'no command'. \
    Input: {human_input}"
)


# 3. LLM Chains
happy_sad_chain = LLMChain(llm=llm, prompt=happy_sad_prompt)
calm_angry_chain = LLMChain(llm=llm, prompt=calm_angry_prompt)
robot_command_chain = LLMChain(llm=llm, prompt=robot_command_prompt)

def rate_sentiment(text, chain, scale_name, current_value):
    """Rates the sentiment of a given text using a Gemini Pro model.
    Args:
        text (str): The text to rate.
        chain (LLMChain): The LLMChain to use
        scale_name (str): The name of the scale to rate
        current_value (int): The current value for this scale
    Returns:
        tuple(str,int): A tuple containing a single number (rating) representing the sentiment between -10 and 10, or "error" if the rating was not in the range
        and the new value of this scale
    """

    result = chain.run(human_input=text)
    
    try:
      rating = int(result.strip())
      if -10 <= rating <= 10:
        current_value += rating
        current_value = max(-10, min(current_value, 10))
        return str(rating), current_value
      else:
        return "error", current_value
    except ValueError:
      return "error", current_value
    
def extract_command(text, chain):
    """
    Extracts a robot command from the given text.
    Args:
        text (str): The text to analyze.
    Returns:
        str: the robot command if found or "no command"
    """
    result = chain.run(human_input=text)

    
    return result.strip()

# Streamlit App
st.title("Sentiment and Emotion Analyzer")

user_input = st.text_area("Enter some text to analyze:")

if user_input:
    happy_sad_rating, sad_value = rate_sentiment(user_input, happy_sad_chain, "happy-sad", sad_value)
    calm_angry_rating, angry_value = rate_sentiment(user_input, calm_angry_chain, "calm-angry", angry_value)
    robot_command = extract_command(user_input, robot_command_chain)

    st.subheader("Analysis Results:")
    if happy_sad_rating != "error":
        st.write(f"Sentiment Rating (Happy-Sad): {happy_sad_rating}")
        st.write(f"Sadness-Happiness Scale: {sad_value}")
    else:
         st.write(f"Error: Couldn't rate the sentiment of this input.")


    if calm_angry_rating != "error":
        st.write(f"Emotion Rating (Calm-Angry): {calm_angry_rating}")
        st.write(f"Anger Scale: {angry_value}")
    else:
        st.write(f"Error: Couldn't rate the emotion of this input.")

    st.write(f"Robot Command: {robot_command}")